In [1]:
# Author: Arthur Mensch
# License: BSD

import matplotlib as mpl
mpl.use('Qt5Agg')

from nilearn.datasets import fetch_atlas_smith_2009
from modl.input_data.fmri.fixes import monkey_patch_nifti_image

monkey_patch_nifti_image()

import matplotlib.pyplot as plt
from sklearn.externals.joblib import Memory
from sklearn.model_selection import train_test_split

from modl.datasets import fetch_adhd
from modl.decomposition.fmri import fMRIDictFact, rfMRIDictionaryScorer
from modl.plotting.fmri import display_maps
from modl.utils.system import get_cache_dirs

/home/sshankar/env/modl/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
n_components = 20
batch_size = 50
learning_rate = .92
method = 'masked'
step_size = 0.01
reduction = 12
alpha = 1e-3
n_epochs = 2
verbose = 15
n_jobs = 2
smoothing_fwhm = 6

In [3]:
dict_init = fetch_atlas_smith_2009().rsn20

In [24]:
dict_init

'/home/sshankar/nilearn_data/smith_2009/rsn20.nii.gz'

In [25]:
from nilearn import image
di = image.load_img(dict_init)
di.shape

(91, 109, 91, 20)

In [4]:
dataset = fetch_adhd(n_subjects=3)
data = dataset.rest.values

/home/sshankar/env/modl/lib/python3.6/site-packages/nilearn/datasets/func.py:503: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


In [5]:
data

array([['/home/sshankar/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz',
        '/home/sshankar/nilearn_data/adhd/data/0010042/0010042_regressors.csv'],
       ['/home/sshankar/nilearn_data/adhd/data/0010064/0010064_rest_tshift_RPI_voreg_mni.nii.gz',
        '/home/sshankar/nilearn_data/adhd/data/0010064/0010064_regressors.csv'],
       ['/home/sshankar/nilearn_data/adhd/data/0010128/0010128_rest_tshift_RPI_voreg_mni.nii.gz',
        '/home/sshankar/nilearn_data/adhd/data/0010128/0010128_regressors.csv']],
      dtype=object)

In [7]:
data[0]

array(['/home/sshankar/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz',
       '/home/sshankar/nilearn_data/adhd/data/0010042/0010042_regressors.csv'],
      dtype=object)

In [5]:
train_data, test_data = train_test_split(data, test_size=1, random_state=0)
train_imgs, train_confounds = zip(*train_data)
test_imgs, test_confounds = zip(*test_data)
mask = dataset.mask
memory = Memory(cachedir=get_cache_dirs()[0],
                verbose=2)

/home/sshankar/env/modl/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/home/sshankar/cache'", use "location='/home/sshankar/cache'" instead.
  


In [22]:
type(train_data)

numpy.ndarray

In [21]:
train_imgs

('/home/sshankar/nilearn_data/adhd/data/0010064/0010064_rest_tshift_RPI_voreg_mni.nii.gz',
 '/home/sshankar/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz')

In [19]:
train_confounds

('/home/sshankar/nilearn_data/adhd/data/0010064/0010064_regressors.csv',
 '/home/sshankar/nilearn_data/adhd/data/0010042/0010042_regressors.csv')

In [6]:
cb = rfMRIDictionaryScorer(test_imgs, test_confounds=test_confounds)
dict_fact = fMRIDictFact(smoothing_fwhm=smoothing_fwhm,
                         method=method,
                         step_size=step_size,
                         mask=mask,
                         memory=memory,
                         memory_level=2,
                         verbose=verbose,
                         n_epochs=n_epochs,
                         n_jobs=n_jobs,
                         random_state=1,
                         n_components=n_components,
                         dict_init=dict_init,
                         positive=True,
                         learning_rate=learning_rate,
                         batch_size=batch_size,
                         reduction=reduction,
                         alpha=alpha,
                         callback=cb,
                         )
dict_fact.fit(train_imgs, confounds=train_confounds)

[MultiNiftiMasker.fit] Loading data from None
[MultiNiftiMasker.transform] Resampling mask
[Memory]7.8s, 0.1min    : Loading resample_img...
________________________________________________________________________________
[Memory] Calling nilearn.masking.unmask...
unmask(array([[-0.805341, ...,  0.      ],
       ...,
       [ 1.61355 , ...,  0.      ]], dtype=float32), 
<modl.input_data.fmri.fixes.Nifti1Image object at 0x7f29ec042f98>)
___________________________________________________________unmask - 0.1s, 0.0min
________________________________________________________________________________
[Memory] Calling modl.decomposition.fmri._compute_components...
_compute_components(MultiNiftiMasker(detrend=True, dtype=None, high_pass=None, low_pass=None,
                 mask_args=None,
                 mask_img='/home/sshankar/modl_data/adhd/mask_img.nii.gz',
                 mask_strategy='background',
                 memory=Memory(location=/home/sshankar/cache/joblib),
                

fMRIDictFact(alpha=0.001, batch_size=50,
             callback=<modl.decomposition.fmri.rfMRIDictionaryScorer object at 0x7f29ec042208>,
             detrend=True,
             dict_init='/home/sshankar/nilearn_data/smith_2009/rsn20.nii.gz',
             high_pass=None, learning_rate=0.92, low_pass=None,
             mask='/home/sshankar/modl_data/adhd/mask_img.nii.gz',
             mask_args=None, mask_strategy='background',
             memory=Memory(location=/home/sshankar/cache/joblib),
             memory_level=2, method='masked', n_components=20, n_epochs=2,
             n_jobs=2, positive=True, random_state=1, reduction=12,
             smoothing_fwhm=6, standardize=True, step_size=0.01, t_r=None,
             target_affine=None, target_shape=None, transform_batch_size=None,
             verbose=15)

In [7]:
fig = plt.figure()
display_maps(fig, dict_fact.components_img_)
fig, ax = plt.subplots(1, 1)
ax.plot(cb.time, cb.score, marker='o')
plt.show()

/home/sshankar/env/modl/lib/python3.6/site-packages/nilearn/plotting/displays.py:98: UserWarning: linewidths is ignored by contourf
  **kwargs)


In [10]:
dict_fact.components_.shape

(20, 212445)